# Thailand Election Data Processing
## Build CONST_RAW and PARTYLIST_RAW datasets for visualization

This notebook processes election66 and election69 data from JSON sources into JavaScript arrays used by the visualization.

### Setup: Import Libraries

In [17]:
import pandas as pd
import numpy as np
import json
import glob
import os
from pathlib import Path
from typing import Dict, List, Any

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


### Configure Paths

In [18]:
# Define data directories
ELECTION66_JSON_DIR = Path.home() / "Documents/GitHub/th-election69-visualization/election66"
ELECTION69_CONST_DIR = Path.home() / "Documents/GitHub/election-69-OCR-result/data/matched/constituency"
ELECTION69_PL_DIR = Path.home() / "Documents/GitHub/election-69-OCR-result/data/matched/party_list"

# Check if paths exist
print(f"Election66 directory exists: {ELECTION66_JSON_DIR.exists()}")
print(f"Election69 constituency directory exists: {ELECTION69_CONST_DIR.exists()}")
print(f"Election69 party_list directory exists: {ELECTION69_PL_DIR.exists()}")

# List available files
if ELECTION66_JSON_DIR.exists():
    print(f"\nElection66 files:")
    for f in sorted(ELECTION66_JSON_DIR.glob("*.json")):
        print(f"  - {f.name}")

Election66 directory exists: True
Election69 constituency directory exists: True
Election69 party_list directory exists: True

Election66 files:
  - th_election66_info_constituency.json
  - th_election66_info_mp_candidate.json
  - th_election66_info_party_overview.json
  - th_election66_info_province.json
  - th_election66_stats_cons.json
  - th_election66_stats_party.json


### Region Mapping Dictionary

In [19]:
# Define region mappings (same as in your notebook)
NE_region = ["อำนาจเจริญ","บึงกาฬ","บุรีรัมย์","ชัยภูมิ","กาฬสินธุ์","ขอนแก่น","เลย",
             "มหาสารคาม","มุกดาหาร","นครพนม","นครราชสีมา","หนองบัวลำภู","หนองคาย",
             "ร้อยเอ็ด","สกลนคร","ศรีสะเกษ","สุรินทร์","อุบลราชธานี","อุดรธานี","ยโสธร"]
N_region  = ["เชียงใหม่","เชียงราย","ลำปาง","ลำพูน","แม่ฮ่องสอน","น่าน","พะเยา","แพร่","อุตรดิตถ์"]
W_region  = ["ตาก","กาญจนบุรี","ราชบุรี","เพชรบุรี","ประจวบคีรีขันธ์"]
E_region  = ["ฉะเชิงเทรา","จันทบุรี","ชลบุรี","ปราจีนบุรี","ระยอง","สระแก้ว","ตราด"]
C_region  = ["อุทัยธานี","อ่างทอง","ชัยนาท","พระนครศรีอยุธยา","ลพบุรี","นครปฐม","นนทบุรี",
             "ปทุมธานี","นครนายก","นครสวรรค์","สมุทรปราการ","สมุทรสาคร","สมุทรสงคราม",
             "สระบุรี","สิงห์บุรี","สุพรรณบุรี","สุโขทัย","พิษณุโลก","พิจิตร","กำแพงเพชร","เพชรบูรณ์"]
BKK       = ["กรุงเทพมหานคร"]
S_region  = ["ชุมพร","นครศรีธรรมราช","นราธิวาส","ปัตตานี","พัทลุง","สงขลา","สุราษฎร์ธานี",
             "ยะลา","กระบี่","พังงา","ภูเก็ต","ระนอง","สตูล","ตรัง"]

region_dict = {
    **{p: "02 ภาคอีสาน"        for p in NE_region},
    **{p: "01 ภาคเหนือ"        for p in N_region},
    **{p: "06 ภาคตะวันตก"      for p in W_region},
    **{p: "03 ภาคตะวันออก"     for p in E_region},
    **{p: "04 ภาคกลาง"         for p in C_region},
    **{p: "05 กรุงเทพมหานคร"   for p in BKK},
    **{p: "07 ภาคใต้"          for p in S_region},
}

print(f"✓ Region mapping defined: {len(region_dict)} provinces")

✓ Region mapping defined: 77 provinces


### Load Election69 Data from JSON

In [20]:
# Cell 1: Import and setup
import json
import pandas as pd
from pathlib import Path

# Cell 2: Paste the load_election66_data function (or import from the .py file)
"""
Notebook-friendly loader for Thai Election 2566 data from JSON files.

Usage in notebook:
    from pathlib import Path
    ELECTION66_JSON_DIR = Path('/mnt/project')  # adjust as needed
    
    voters66 = load_election66_data(ELECTION66_JSON_DIR)
"""

import json
import pandas as pd
from pathlib import Path


def load_election66_data(json_dir, verbose=True):
    """
    Load election 2566 data from the set of JSON files.
    
    Expected files in json_dir:
    - th_election66_info_constituency.json
    - th_election66_info_province.json  
    - th_election66_info_party_overview.json
    - th_election66_stats_cons.json
    
    Returns:
        pd.DataFrame with 400 rows (one per constituency) and columns matching
        the th_election66_constituency_results.csv structure.
    """
    json_dir = Path(json_dir)
    
    # Define file paths
    files = {
        'cons_info': json_dir / 'th_election66_info_constituency.json',
        'province': json_dir / 'th_election66_info_province.json',
        'party': json_dir / 'th_election66_info_party_overview.json',
        'stats': json_dir / 'th_election66_stats_cons.json',
    }
    
    # Check files exist
    missing = [k for k, v in files.items() if not v.exists()]
    if missing:
        print(f"⚠ Missing files: {missing}")
        return pd.DataFrame()
    
    # Load JSON files
    def load_json(path):
        with open(path, 'r', encoding='utf-8-sig') as f:
            return json.load(f)
    
    cons_info_data = load_json(files['cons_info'])
    province_data = load_json(files['province'])
    party_data = load_json(files['party'])
    stats_data = load_json(files['stats'])
    
    # Build lookup dictionaries
    # Province: prov_id -> {thai, eng}
    province_lookup = {
        p['prov_id']: {'thai': p['province'], 'eng': p['eng']}
        for p in province_data['province']
    }
    
    # Party: party_id (int) -> party_name
    party_lookup = {int(p['id']): p['name'] for p in party_data}
    
    # Constituency info: cons_id -> {cons_no, prov_id, registered_vote}
    cons_info_lookup = {
        c['cons_id']: {
            'cons_no': c['cons_no'],
            'prov_id': c['prov_id'],
            'registered_vote': c['registered_vote']
        }
        for c in cons_info_data
    }
    
    if verbose:
        print(f"✓ Loaded {len(province_lookup)} provinces, {len(party_lookup)} parties, {len(cons_info_lookup)} constituencies")
    
    # Process constituencies
    rows = []
    
    for province in stats_data['result_province']:
        prov_id = province['prov_id']
        prov_names = province_lookup.get(prov_id, {'thai': '', 'eng': ''})
        
        for cons in province.get('constituencies', []):
            cons_id = cons['cons_id']
            info = cons_info_lookup.get(cons_id, {})
            
            # Get top 3 candidates by rank
            candidates = sorted(cons.get('candidates', []), key=lambda x: x.get('mp_app_rank', 999))
            
            def get_candidate(idx):
                if idx < len(candidates):
                    c = candidates[idx]
                    return c['party_id'], party_lookup.get(c['party_id'], 'Unknown'), int(c.get('mp_app_vote', 0))
                return None, None, None
            
            winner_party_id, winner_party, winner_votes = get_candidate(0)
            runnerup_party_id, runnerup_party, runnerup_votes = get_candidate(1)
            third_party_id, third_party, third_votes = get_candidate(2)
            
            # Get top 2 party list results
            result_party = sorted(cons.get('result_party', []), key=lambda x: x.get('party_list_vote', 0), reverse=True)
            
            def get_pl_party(idx):
                if idx < len(result_party):
                    p = result_party[idx]
                    return p['party_id'], party_lookup.get(p['party_id'], 'Unknown'), p.get('party_list_vote', 0)
                return None, None, None
            
            pl_winner_party_id, pl_winner_party, pl_winner_votes = get_pl_party(0)
            pl_runnerup_party_id, pl_runnerup_party, pl_runnerup_votes = get_pl_party(1)
            
            # Calculate derived fields
            valid_votes = cons.get('valid_votes', 0)
            margin_votes = (winner_votes - runnerup_votes) if winner_votes and runnerup_votes else 0
            margin_pct = (margin_votes / valid_votes * 100) if valid_votes > 0 else 0.0
            
            turn_out = cons.get('turn_out', 0)
            pl_turn_out = cons.get('party_list_turn_out', 0)
            
            rows.append({
                'cons_id': cons_id,
                'prov_id': prov_id,
                'province_thai': prov_names['thai'],
                'province_eng': prov_names['eng'],
                'cons_no': info.get('cons_no', 0),
                'registered_vote': info.get('registered_vote', 0),
                'turn_out': turn_out,
                'percent_turn_out': cons.get('percent_turn_out', 0.0),
                'valid_votes': valid_votes,
                'invalid_votes': cons.get('invalid_votes', 0),
                'blank_votes': cons.get('blank_votes', 0),
                'percent_valid': cons.get('percent_valid_votes', 0.0),
                'percent_invalid': cons.get('percent_invalid_votes', 0.0),
                'winner_party_id': winner_party_id,
                'winner_party': winner_party,
                'winner_votes': winner_votes,
                'runnerup_party_id': runnerup_party_id,
                'runnerup_party': runnerup_party,
                'runnerup_votes': runnerup_votes,
                'third_party_id': third_party_id,
                'third_party': third_party,
                'third_votes': third_votes,
                'margin_votes': margin_votes,
                'margin_pct': round(margin_pct, 4),
                'total_candidates': len(candidates),
                'pl_turn_out': pl_turn_out,
                'pl_percent_turn_out': cons.get('party_list_percent_turn_out', 0.0),
                'pl_valid_votes': cons.get('party_list_valid_votes', 0),
                'pl_invalid_votes': cons.get('party_list_invalid_votes', 0),
                'pl_blank_votes': cons.get('party_list_blank_votes', 0),
                'pl_percent_valid': cons.get('party_list_percent_valid_votes', 0.0),
                'pl_percent_invalid': cons.get('party_list_percent_invalid_votes', 0.0),
                'pl_winner_party_id': pl_winner_party_id,
                'pl_winner_party': pl_winner_party,
                'pl_winner_votes': pl_winner_votes,
                'pl_runnerup_party_id': pl_runnerup_party_id,
                'pl_runnerup_party': pl_runnerup_party,
                'pl_runnerup_votes': pl_runnerup_votes,
                'cons_pl_turnout_diff': turn_out - pl_turn_out,
            })
    
    df = pd.DataFrame(rows).sort_values('cons_id').reset_index(drop=True)
    
    if verbose:
        print(f"✓ Loaded {len(df)} constituency records")
        print(df.head())
    
    return df


# =============================================================================
# Additional helper: Load party color mapping
# =============================================================================

def load_party_colors(json_dir):
    """
    Load party color dictionary from th_election66_info_party_overview.json
    
    Returns:
        dict: party_name -> color (hex string)
    """
    json_dir = Path(json_dir)
    fpath = json_dir / 'th_election66_info_party_overview.json'
    
    with open(fpath, 'r', encoding='utf-8-sig') as f:
        party_data = json.load(f)
    
    return {p['name']: p['color'] for p in party_data}


# =============================================================================
# Quick test when run directly
# =============================================================================

if __name__ == '__main__':
    # Test with project directory
    df = load_election66_data('/mnt/project')
    print(f"\nShape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

# Cell 3: Load the data
voters66 = load_election66_data(ELECTION66_JSON_DIR)

# Also load party colors for visualization
party_color_dict = load_party_colors(ELECTION66_JSON_DIR)
voters66

⚠ Missing files: ['cons_info', 'province', 'party', 'stats']

Shape: (0, 0)
Columns: []
✓ Loaded 77 provinces, 70 parties, 400 constituencies
✓ Loaded 400 constituency records
  cons_id prov_id    province_thai              province_eng  cons_no  \
0   ACR_1     ACR       อำนาจเจริญ             AMNAT CHAROEN        1   
1   ACR_2     ACR       อำนาจเจริญ             AMNAT CHAROEN        2   
2   ATG_1     ATG          อ่างทอง                 ANG THONG        1   
3   ATG_2     ATG          อ่างทอง                 ANG THONG        2   
4   AYA_1     AYA  พระนครศรีอยุธยา  PHRA NAKHON SI AYUTTHAYA        1   

   registered_vote  turn_out  percent_turn_out  valid_votes  invalid_votes  \
0           145416    105574          72.60136       100946           3359   
1           155936    111813          71.70442       106888           3824   
2           111355     89918          80.74896        84595           3263   
3           112719     91466          81.14515        86144           380

,cons_id,prov_id,province_thai,province_eng,cons_no,registered_vote,turn_out,percent_turn_out,valid_votes,invalid_votes,...,pl_blank_votes,pl_percent_valid,pl_percent_invalid,pl_winner_party_id,pl_winner_party,pl_winner_votes,pl_runnerup_party_id,pl_runnerup_party,pl_runnerup_votes,cons_pl_turnout_diff
0,ACR_1,ACR,อำนาจเจริญ,AMNAT CHAROEN,1,145416,105574,72.60136,100946,3359,...,681,96.00186,3.35215,726,ก้าวไกล,33892,705,เพื่อไทย,33576,0
1,ACR_2,ACR,อำนาจเจริญ,AMNAT CHAROEN,2,155936,111813,71.70442,106888,3824,...,722,95.66061,3.69367,705,เพื่อไทย,36248,726,ก้าวไกล,35581,0
2,ATG_1,ATG,อ่างทอง,ANG THONG,1,111355,89918,80.74896,84595,3263,...,1306,94.60953,3.93581,726,ก้าวไกล,35131,705,เพื่อไทย,26909,0
3,ATG_2,ATG,อ่างทอง,ANG THONG,2,112719,91466,81.14515,86144,3805,...,1008,94.59143,4.30652,705,เพื่อไทย,32038,726,ก้าวไกล,31427,0
4,AYA_1,AYA,พระนครศรีอยุธยา,PHRA NAKHON SI AYUTTHAYA,1,138032,110317,79.92132,102821,3578,...,2310,94.60373,3.30230,726,ก้าวไกล,49110,705,เพื่อไทย,23913,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,YLA_2,YLA,ยะลา,YALA,2,129255,100319,77.61325,93349,4274,...,1303,92.45008,6.25106,740,ประชาชาติ,55426,726,ก้าวไกล,19011,0
396,YLA_3,YLA,ยะลา,YALA,3,127445,97956,76.86139,90913,4229,...,1529,93.24799,5.19111,740,ประชาชาติ,40863,726,ก้าวไกล,21094,0
397,YST_1,YST,ยโสธร,YASOTHON,1,144816,110491,76.29751,106293,2857,...,712,96.35720,2.99750,705,เพื่อไทย,42534,726,ก้าวไกล,33949,-1
398,YST_2,YST,ยโสธร,YASOTHON,2,141902,98432,69.36618,94418,3161,...,404,96.72159,2.86797,705,เพื่อไทย,52600,726,ก้าวไกล,24010,0


In [21]:
import json
import pandas as pd
from pathlib import Path

ELECTION66_JSON_DIR = Path('/path/to/your/json/files')

In [22]:
def load_json_folder(folder, ballot_type="constituency"):
    """Load election results from JSON folder (as per your notebook)"""
    rows = []
    
    json_files = sorted(glob.glob(os.path.join(folder, "*.json")))
    print(f"Found {len(json_files)} {ballot_type} files")
    
    for fpath in json_files:
        with open(fpath, encoding="utf-8") as f:
            d = json.load(f)

        province    = d["province_name_normalized"]
        cons_no     = d["constituency_number"]
        summary     = d["summary"]
        results     = d["results"]

        # Sort by votes descending 
        sorted_results = sorted(results, key=lambda x: x["votes"], reverse=True)
        winner   = sorted_results[0]  if len(sorted_results) > 0 else {"party": None, "votes": 0}
        runnerup = sorted_results[1]  if len(sorted_results) > 1 else {"party": None, "votes": 0}

        total_valid = summary["good_votes"]
        others = total_valid - winner["votes"] - runnerup["votes"]

        rows.append({
            "province":           province,
            "constituency_number": cons_no,
            "total_valid":        total_valid,
            "invalid_ballots":    summary["invalid_votes"],
            "no_votes":           summary["no_votes"],
            "voters_came":        summary["voters_came"],
            "winning_score":      winner["votes"],
            "winning_party":      winner["party"],
            "runnerUp_score":     runnerup["votes"],
            "runnerUp_party":     runnerup["party"],
            "others_score":       max(others, 0),
        })
    return pd.DataFrame(rows)

# Load data
if ELECTION69_CONST_DIR.exists():
    cons_df = load_json_folder(str(ELECTION69_CONST_DIR), "constituency")
    print(f"Loaded {len(cons_df)} constituency records")
    print(cons_df.head())
else:
    print("⚠ Election69 constituency data directory not found")
    cons_df = pd.DataFrame()

Found 387 constituency files
Loaded 387 constituency records
        province  constituency_number  total_valid  invalid_ballots  no_votes  \
0  กรุงเทพมหานคร                    1        77075             1146      4200   
1  กรุงเทพมหานคร                   10        91913             1521      6223   
2  กรุงเทพมหานคร                   11        94994             1207      4167   
3  กรุงเทพมหานคร                   12       103439             1601      5989   
4  กรุงเทพมหานคร                   13        90508             1315      5242   

   voters_came  winning_score winning_party  runnerUp_score runnerUp_party  \
0        82421          34167       ประชาชน           14813   ประชาธิปัตย์   
1        99657          41804       ประชาชน           19047       เพื่อไทย   
2       100367          38779       ประชาชน           24850       เพื่อไทย   
3       111029          49925       ประชาชน           16106      ภูมิใจไทย   
4        97065          44511       ประชาชน           15227   

In [23]:
if ELECTION69_PL_DIR.exists():
    pl_df = load_json_folder(str(ELECTION69_PL_DIR), "party_list")
    print(f"\nLoaded {len(pl_df)} party_list records")
    print(pl_df.head())
else:
    print("⚠ Election69 party_list data directory not found")
    pl_df = pd.DataFrame()

Found 386 party_list files

Loaded 386 party_list records
        province  constituency_number  total_valid  invalid_ballots  no_votes  \
0  กรุงเทพมหานคร                    1        78517             1726      2178   
1  กรุงเทพมหานคร                   10        94890             1779      2987   
2  กรุงเทพมหานคร                   11        96157             1769      2486   
3  กรุงเทพมหานคร                   12       106754             1499      2776   
4  กรุงเทพมหานคร                   13        93463             1380      2222   

   voters_came  winning_score winning_party  runnerUp_score runnerUp_party  \
0        82421          34215       ประชาชน           16471      ภูมิใจไทย   
1        99657          44757       ประชาชน           16002      ภูมิใจไทย   
2       100367          44980       ประชาชน           18501      ภูมิใจไทย   
3       111029          52294       ประชาชน           19124      ภูมิใจไทย   
4        97065          44989       ประชาชน           17977      

### Merge Election69 Data (Constituency + Party List)

In [24]:
# Merge on province + constituency number
if not cons_df.empty and not pl_df.empty:
    voters69 = pd.merge(
        cons_df.rename(columns=lambda c: f"const_{c}" if c not in ("province","constituency_number") else c),
        pl_df.rename(columns=lambda c: f"party_{c}" if c not in ("province","constituency_number") else c),
        on=["province", "constituency_number"]
    )
    
    # Calculate ballot difference
    voters69["voters_diff_const_PL"] = (
        (voters69["const_total_valid"] + voters69["const_invalid_ballots"] + voters69["const_no_votes"]) -
        (voters69["party_total_valid"] + voters69["party_invalid_ballots"] + voters69["party_no_votes"])
    )
    
    # Add region
    voters69["region"] = voters69["province"].map(region_dict)
    
    print(f"✓ Merged {len(voters69)} records")
    print(f"\nData structure:")
    print(voters69.info())
    print(f"\nSample:")
    print(voters69.head())
else:
    voters69 = pd.DataFrame()
    print("⚠ Cannot merge - one or both dataframes are empty")

✓ Merged 385 records

Data structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   province               385 non-null    object
 1   constituency_number    385 non-null    int64 
 2   const_total_valid      385 non-null    int64 
 3   const_invalid_ballots  385 non-null    int64 
 4   const_no_votes         385 non-null    int64 
 5   const_voters_came      385 non-null    int64 
 6   const_winning_score    385 non-null    int64 
 7   const_winning_party    385 non-null    object
 8   const_runnerUp_score   385 non-null    int64 
 9   const_runnerUp_party   385 non-null    object
 10  const_others_score     385 non-null    int64 
 11  party_total_valid      385 non-null    int64 
 12  party_invalid_ballots  385 non-null    int64 
 13  party_no_votes         385 non-null    int64 
 14  party_voters_came      385 non-null 

### Transform to CONST_RAW Format

In [25]:
# Cell 1: Debug - see what's not matching
print("=== Provinces in voters66 ===")
print(sorted(voters66['province_thai'].unique())[:10])

print("\n=== Provinces in voters69 ===")
print(sorted(voters69['province'].unique())[:10])

# Check overlap
prov66 = set(voters66['province_thai'].unique())
prov69 = set(voters69['province'].unique())
print(f"\nMatched: {len(prov66 & prov69)}, Only in 66: {len(prov66 - prov69)}, Only in 69: {len(prov69 - prov66)}")

=== Provinces in voters66 ===
['กระบี่', 'กรุงเทพมหานคร', 'กาญจนบุรี', 'กาฬสินธุ์', 'กำแพงเพชร', 'ขอนแก่น', 'จันทบุรี', 'ฉะเชิงเทรา', 'ชลบุรี', 'ชัยนาท']

=== Provinces in voters69 ===
['กระบี่', 'กรุงเทพมหานคร', 'กาญจนบุรี', 'กาฬสินธุ์', 'กำแพงเพชร', 'ขอนแก่น', 'จันทบุรี', 'ฉะเชิงเทรา', 'ชลบุรี', 'ชัยนาท']

Matched: 76, Only in 66: 1, Only in 69: 0


In [26]:
# Find the missing province
prov66 = set(voters66['province_thai'].unique())
prov69 = set(voters69['province'].unique())

print("Province only in 2566 (no 2569 data):")
print(prov66 - prov69)

print("\nProvince only in 2569 (no 2566 data):")
print(prov69 - prov66)

Province only in 2566 (no 2569 data):
{'บุรีรัมย์'}

Province only in 2569 (no 2566 data):
set()


In [27]:
# Check constituency counts per province
cons66 = voters66.groupby('province_thai')['cons_no'].nunique().sort_index()
cons69 = voters69.groupby('province')['constituency_number'].nunique().sort_index()

# Compare
comparison = pd.DataFrame({
    'cons_2566': cons66,
    'cons_2569': cons69
}).fillna(0).astype(int)

comparison['diff'] = comparison['cons_2569'] - comparison['cons_2566']
print("Provinces with different constituency counts:")
print(comparison[comparison['diff'] != 0])

print(f"\nTotal constituencies: 2566={cons66.sum()}, 2569={cons69.sum()}")

Provinces with different constituency counts:
               cons_2566  cons_2569  diff
กรุงเทพมหานคร         33         32    -1
นครพนม                 4          3    -1
นครศรีธรรมราช         10          9    -1
น่าน                   3          2    -1
บุรีรัมย์             10          0   -10
ปทุมธานี               7          8     1
ปราจีนบุรี             3          2    -1
ลพบุรี                 5          4    -1
สมุทรสาคร              3          4     1
อุดรธานี              10          9    -1

Total constituencies: 2566=400, 2569=385


In [28]:
# Complete merge with safe NaN handling
voters_merged = pd.merge(
    voters69,
    voters66.rename(columns={
        'province_thai': "province",
        "invalid_votes": "invalid_votes_2566", 
        "percent_invalid": "percent_invalid_2566",
        "turn_out": "turn_out_2566",
        "valid_votes": "valid_votes_2566",
        "winner_party": "winner_party_2566",
        "winner_votes": "winner_votes_2566",
        "runnerup_votes": "runnerup_votes_2566",
        "margin_votes": "margin_votes_2566",
    })[['province', 'cons_no', 'invalid_votes_2566', 'percent_invalid_2566', 
        'turn_out_2566', 'valid_votes_2566', 'winner_party_2566', 
        'winner_votes_2566', 'runnerup_votes_2566', 'margin_votes_2566']],
    left_on=["province", "constituency_number"],
    right_on=["province", "cons_no"],
    how="left"
).drop(columns=['cons_no'])

# Fill NaN with 0 for numeric columns, "Unknown" for strings
numeric_cols_2566 = ['invalid_votes_2566', 'percent_invalid_2566', 'turn_out_2566', 
                     'valid_votes_2566', 'winner_votes_2566', 'runnerup_votes_2566', 'margin_votes_2566']
voters_merged[numeric_cols_2566] = voters_merged[numeric_cols_2566].fillna(0)
voters_merged['winner_party_2566'] = voters_merged['winner_party_2566'].fillna("Unknown")

# Flag rows without 2566 comparison data
voters_merged['has_2566_data'] = voters_merged['turn_out_2566'] > 0

print(f"✓ Merged: {len(voters_merged)} records")
print(f"  With 2566 comparison: {voters_merged['has_2566_data'].sum()}")
print(f"  Without 2566 data: {(~voters_merged['has_2566_data']).sum()}")

✓ Merged: 385 records
  With 2566 comparison: 383
  Without 2566 data: 2


In [29]:
# Create province name mapping from voters66
province_name_map = voters66[['province_thai', 'province_eng']].drop_duplicates()
province_name_map = dict(zip(province_name_map['province_thai'], province_name_map['province_eng']))

prov_id_name_map = voters66[['province_thai', 'prov_id']].drop_duplicates()
prov_id_name_map = dict(zip(prov_id_name_map['province_thai'], prov_id_name_map['prov_id']))
# Add to voters69
voters69['province_eng'] = voters69['province'].map(province_name_map)
voters69['prov_id'] = voters69['province'].map(prov_id_name_map)

# Check for any unmapped
unmapped = voters69[voters69['province_eng'].isna()]['province'].unique()
unmapped2 = voters69[voters69['prov_id'].isna()]['province'].unique()
if len(unmapped) > 0:
    print(f"⚠ Unmapped provinces: {unmapped}")
    print(f"⚠ Unmapped provinces: {unmapped2}")
else:
    print(f"✓ Added province_eng to all {len(voters69)} records")
    print(f"✓ Added province_eng to all {len(voters69)} records")

✓ Added province_eng to all 385 records
✓ Added province_eng to all 385 records


In [30]:
# ============================================================
# Build CONST_RAW and PARTYLIST_RAW with all required fields
# ============================================================

import json
import os

# Create province name mapping from voters66
province_name_map = voters66[['province_thai', 'province_eng', 'prov_id']].drop_duplicates()
province_to_eng = dict(zip(province_name_map['province_thai'], province_name_map['province_eng']))
province_to_id = dict(zip(province_name_map['province_thai'], province_name_map['prov_id']))

# Safe conversion functions
def safe_int(val, default=0):
    if pd.isna(val):
        return default
    try:
        return int(val)
    except (ValueError, TypeError):
        return default

def safe_float(val, default=0.0):
    if pd.isna(val):
        return default
    try:
        return float(val)
    except (ValueError, TypeError):
        return default

def safe_str(val, default="Unknown"):
    if pd.isna(val) or val is None:
        return default
    return str(val)


In [31]:
# ============================================================
# Build CONST_RAW and PARTYLIST_RAW with correct surplus calculation
# ============================================================

CONST_RAW = []
PARTYLIST_RAW = []

for idx, row in voters_merged.iterrows():
    province = row.get("province", "Unknown")
    
    # ── Constituency (Green/เขียว) data ──
    const_invalid = safe_int(row.get("const_invalid_ballots"))
    const_voters = safe_int(row.get("const_voters_came"))
    const_valid = safe_int(row.get("const_total_valid"))
    const_no_votes = safe_int(row.get("const_no_votes"))
    const_winning = safe_int(row.get("const_winning_score"))
    const_runnerup = safe_int(row.get("const_runnerUp_score"))
    const_others = max(0, const_valid - const_winning - const_runnerup)
    const_total_used = const_valid + const_invalid + const_no_votes
    
    # ── Party List (Pink/ชมพู) data ──
    party_invalid = safe_int(row.get("party_invalid_ballots"))
    party_voters = safe_int(row.get("party_voters_came"))
    party_valid = safe_int(row.get("party_total_valid"))
    party_no_votes = safe_int(row.get("party_no_votes"))
    party_winning = safe_int(row.get("party_winning_score"))
    party_runnerup = safe_int(row.get("party_runnerUp_score"))
    party_others = max(0, party_valid - party_winning - party_runnerup)
    party_total_used = party_valid + party_invalid + party_no_votes
    
    # ── Ballot surplus (Green - Pink) ──
    # Positive = more green ballots, Negative = more pink ballots
    ballot_surplus = const_total_used - party_total_used
    
    # ── 2566 data ──
    invalid_2566 = safe_int(row.get("invalid_votes_2566"))
    percent_invalid_2566 = safe_float(row.get("percent_invalid_2566"))
    turnout_2566 = safe_int(row.get("turn_out_2566"))
    winner_2566 = safe_str(row.get("winner_party_2566"))
    
    # ── Calculated fields ──
    const_invalid_pct = (const_invalid / const_total_used * 100) if const_total_used > 0 else 0
    const_margin = const_winning - const_runnerup
    party_invalid_pct = (party_invalid / party_total_used * 100) if party_total_used > 0 else 0
    party_margin = party_winning - party_runnerup
    
    # ── Common location fields ──
    location = {
        "province_thai": province,
        "province_eng": province_to_eng.get(province, province),
        "prov_id": province_to_id.get(province, province[:3].upper()),
        "cons_no": int(row.get("constituency_number", 0)),
        "region": safe_str(row.get("region")),
    }
    
    # ── CONST_RAW record ──
    const_record = {
        **location,
        
        # 2566 data
        "percent_invalid_2566": round(percent_invalid_2566, 2),
        "turn_out_2566": turnout_2566,
        "winner_party_2566": winner_2566,
        
        # 2569 constituency breakdown
        "turn_out_2569": const_voters,
        "total_used_2569": const_total_used,
        "valid_2569": const_valid,
        "invalid_2569": const_invalid,
        "blank_2569": const_no_votes,
        "percent_invalid_2569": round(const_invalid_pct, 2),
        "winner_party_2569": safe_str(row.get("const_winning_party")),
        "winner_votes_2569": const_winning,
        "runnerup_party_2569": safe_str(row.get("const_runnerUp_party")),
        "runnerup_votes_2569": const_runnerup,
        "others_2569": const_others,
        "margin_2569": const_margin,
        
        # Ballot surplus (const - party list)
        "ballot_surplus": ballot_surplus,  # positive = green surplus, negative = pink surplus
        
        # Changes
        "invalid_change": const_invalid - invalid_2566,
        "invalid_pct_change": round(const_invalid_pct - percent_invalid_2566, 3),
        "turnout_change": const_voters - turnout_2566,
    }
    CONST_RAW.append(const_record)
    
    # ── PARTYLIST_RAW record ──
    party_record = {
        **location,
        
        # 2566 data (same baseline)
        "percent_invalid_2566": round(percent_invalid_2566, 2),
        "turn_out_2566": turnout_2566,
        "winner_party_2566": winner_2566,
        
        # 2569 party list breakdown
        "turn_out_2569": party_voters,
        "total_used_2569": party_total_used,
        "valid_2569": party_valid,
        "invalid_2569": party_invalid,
        "blank_2569": party_no_votes,
        "percent_invalid_2569": round(party_invalid_pct, 2),
        "winner_party_2569": safe_str(row.get("party_winning_party")),
        "winner_votes_2569": party_winning,
        "runnerup_party_2569": safe_str(row.get("party_runnerUp_party")),
        "runnerup_votes_2569": party_runnerup,
        "others_2569": party_others,
        "margin_2569": party_margin,
        
        # Ballot surplus (same value, but from party list perspective it's negated)
        "ballot_surplus": ballot_surplus,  # same value - positive = green surplus
        
        # Changes
        "invalid_change": party_invalid - invalid_2566,
        "invalid_pct_change": round(party_invalid_pct - percent_invalid_2566, 3),
    }
    PARTYLIST_RAW.append(party_record)

# ── Summary ──
print(f"✓ Built CONST_RAW: {len(CONST_RAW)} records")
print(f"✓ Built PARTYLIST_RAW: {len(PARTYLIST_RAW)} records")

# Check surplus distribution
surplus_positive = sum(1 for r in CONST_RAW if r['ballot_surplus'] > 0)
surplus_negative = sum(1 for r in CONST_RAW if r['ballot_surplus'] < 0)
surplus_zero = sum(1 for r in CONST_RAW if r['ballot_surplus'] == 0)
total_surplus = sum(r['ballot_surplus'] for r in CONST_RAW)

print(f"\nBallot surplus distribution:")
print(f"  บัตรเขียว > บัตรชมพู: {surplus_positive} เขต")
print(f"  บัตรชมพู > บัตรเขียว: {surplus_negative} เขต")
print(f"  เท่ากัน: {surplus_zero} เขต")
print(f"  รวมส่วนต่างทั้งหมด: {total_surplus:+,} บัตร")

✓ Built CONST_RAW: 385 records
✓ Built PARTYLIST_RAW: 385 records

Ballot surplus distribution:
  บัตรเขียว > บัตรชมพู: 91 เขต
  บัตรชมพู > บัตรเขียว: 72 เขต
  เท่ากัน: 222 เขต
  รวมส่วนต่างทั้งหมด: +553 บัตร


In [32]:
# ============================================================
# Export to JavaScript file
# ============================================================
output_path = "/home/ronnie-rattan/Documents/GitHub/th-election69-visualization/election_data_generated.js"

js_content = f"""// Auto-generated election data
// Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
// Constituencies: {len(CONST_RAW)}
// Party List: {len(PARTYLIST_RAW)}

const CONST_RAW = {json.dumps(CONST_RAW, ensure_ascii=False, indent=2)};

const PARTYLIST_RAW = {json.dumps(PARTYLIST_RAW, ensure_ascii=False, indent=2)};
"""

with open(output_path, 'w', encoding='utf-8') as f:
    f.write(js_content)

print(f"\n✓ Exported to: {output_path}")
print(f"  CONST_RAW: {len(CONST_RAW)} records")
print(f"  PARTYLIST_RAW: {len(PARTYLIST_RAW)} records")
print(f"  File size: {os.path.getsize(output_path) / 1024:.1f} KB")


✓ Exported to: /home/ronnie-rattan/Documents/GitHub/th-election69-visualization/election_data_generated.js
  CONST_RAW: 385 records
  PARTYLIST_RAW: 385 records
  File size: 595.2 KB
